# This app focuses on extracting cryptocurrencies data from coingecko's API

## APIs used:

    - Get/coins/markets to get a list of all supported coins, ath, price, volume and other info
    - Get/coins/{id}/history to get community and developer data


In [1]:
import requests
import pandas as pd

In [66]:
# Find an API that has information on coin
# Create an empty list to hold the coins information
crypto_list = []
# Make a loop to get 10 pages
for i in range(1,10,1):
    page_number = i
    base_url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page={page_number}&sparkline=false"
    print(base_url)

    # Request json file from the url
    response_json = requests.get(base_url).json()

    # checkout response
    crypto_list.append(response_json)

https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=1&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=2&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=3&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=4&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=5&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=6&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=7&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=8&sparkline=false
https://api.coingecko.com/api/v3

In [147]:
previous_df = pd.DataFrame(crypto_list[0])
for i in range(len(crypto_list)):
    next_df = pd.DataFrame(crypto_list[i])
    complete_df = pd.concat([previous_df, next_df])
    previous_df = complete_df



In [157]:
clean_complete_df = complete_df.reset_index(drop=True)
#clean_complete_df

In [158]:
# See the name of the columns
#clean_complete_df.columns

In [159]:
# Only get specific columns
crypto_df = clean_complete_df[['id', 'symbol', 'current_price','market_cap', 'market_cap_rank', 'ath', 'ath_change_percentage', 'ath_date','atl', 'atl_change_percentage', 'atl_date']]

In [160]:
# Only run this cell if the data hasn't been renamed and cleaned
if crypto_df.columns[3] == "market_cap":
    # Clean data by dropping columns, ONLY RUN IT ONCE, since columns will change name
    #crypto_df = crypto_df[['id', 'symbol', 'current_price','market_cap', 'market_cap_rank', 'ath', 'ath_change_percentage', 'ath_date','atl', 'atl_change_percentage', 'atl_date']]
    crypto_df['market_cap'] = crypto_df['market_cap']/1000000000

    # Rename columns to see easier
    crypto_df.rename(columns={'market_cap':'market_cap_billion(2021)',
                              'current_price' : 'price_04-22-21'

                             }, inplace=True)


<ipython-input-160-3d38269c4a46>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crypto_df['market_cap'] = crypto_df['market_cap']/1000000000
/Users/johannvillalvir/opt/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [161]:
crypto_df.head()

,id,symbol,price_04-22-21,market_cap_billion(2021),market_cap_rank,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date
0,bitcoin,btc,50559.00,944.198553,1,64805.00,-22.55285,2021-04-14T11:54:46.763Z,67.810000,7.391586e+04,2013-07-06T00:00:00.000Z
1,ethereum,eth,2334.21,269.310585,2,2640.37,-12.70651,2021-04-22T16:34:50.091Z,0.432979,5.322284e+05,2015-10-20T00:00:00.000Z
2,binancecoin,bnb,515.35,79.301086,3,610.06,-16.66939,2021-04-12T09:32:32.635Z,0.039818,1.276626e+06,2017-10-19T00:00:00.000Z
3,tether,usdt,1.01,49.708914,5,1.32,-23.98225,2018-07-24T00:00:00.000Z,0.572521,7.567693e+01,2015-03-02T00:00:00.000Z
4,ripple,xrp,1.09,49.693891,4,3.40,-68.20636,2018-01-07T00:00:00.000Z,0.002686,4.012367e+04,2014-05-22T00:00:00.000Z


In [209]:
# Get each coin's price on 01-04-2018 to now
# Make empty list to hold the value of coins

coin_data_2018 = []

target_date = "01-04-2018"

In [210]:
# Make function to perform request based on specific data, use dataframe above's id to search that specific coin
def obtain_metrics_by_date (target_list = coin_data_2018, date='01-04-2018'):
    # Set up error counter to break out if there are too many errors
    error_counter = 0
    for index, row in crypto_df.iterrows():
        target_coin = row['id']
        print(f"Processing {target_coin}")
           
        if error_counter < 15:
            try:
                # build url
                price_url = f"https://api.coingecko.com/api/v3/coins/{target_coin}/history?date={date}localization=False"
                print(price_url)
                # save json respons
                response_json = requests.get(price_url).json()
                # traverse json getting specific data
                coin_id = response_json['id']
                coin_price = response_json['market_data']['current_price']['usd']
                coin_reddit_subscribers = response_json['community_data']['reddit_subscribers']
                coin_twitter_followers = response_json['community_data']['twitter_followers']
                coin_alexa_rank = response_json['public_interest_stats']['alexa_rank']
                # Add data to empty list in dictionary style
                target_list.append({
                    "id" : coin_id,
                    f"price {date}" : coin_price,
                    f"reddit_subscribers {date}" : coin_reddit_subscribers,
                    f"twitter_followers {date}" : coin_twitter_followers,
                    f"alexa_rank {date}" : coin_alexa_rank
                       })
            # If there's a key error, it means there's no market data for that coin
            except :
                try:
                    #print message 
                    print("Key error. Coin too young to find all data")
                    error_counter+=1
                    #still add the coin to the list
                    target_list.append({'id': coin_id})
                ## Somehow I stopped getting timed out after adding this code, so the code never gets here, but keep it
                except JSONDecodeError :
                    print('Ruh Roh! Looks like you have been timed out')
                    print(f"The last coin to be pulled was {coin_id} at index {index}")
        else:
            print(f"You have been timed out at index: {index}, coin: {target_coin}")
            break
    print("Done with Data Retieval")
    print(f"We found a total of {len(target_list)} entries")



In [208]:
obtain_metrics_by_date()

Processing bitcoin
https://api.coingecko.com/api/v3/coins/bitcoin/history?date=01-04-2018localization=False
Processing ethereum
https://api.coingecko.com/api/v3/coins/ethereum/history?date=01-04-2018localization=False
Processing binancecoin
https://api.coingecko.com/api/v3/coins/binancecoin/history?date=01-04-2018localization=False
Processing tether
https://api.coingecko.com/api/v3/coins/tether/history?date=01-04-2018localization=False
Processing ripple
https://api.coingecko.com/api/v3/coins/ripple/history?date=01-04-2018localization=False
Processing cardano
https://api.coingecko.com/api/v3/coins/cardano/history?date=01-04-2018localization=False
Processing polkadot
https://api.coingecko.com/api/v3/coins/polkadot/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
Processing dogecoin
https://api.coingecko.com/api/v3/coins/dogecoin/history?date=01-04-2018localization=False
Processing uniswap
https://api.coingecko.com/api/v3/coins/uniswap/history?date=01-0

In [202]:
coin_data_2018


[{'id': 'bitcoin',
  'price 01-04-2018': 6975.27531402076,
  'reddit_subscribers 01-04-2018': 785619,
  'twitter_followers 01-04-2018': 819312,
  'alexa_rank 01-04-2018': 5574},
 {'id': 'ethereum',
  'price 01-04-2018': 395.79638095782116,
  'reddit_subscribers 01-04-2018': 348116,
  'twitter_followers 01-04-2018': 382159,
  'alexa_rank 01-04-2018': 8204},
 {'id': 'binancecoin',
  'price 01-04-2018': 11.115510878254739,
  'reddit_subscribers 01-04-2018': 40605,
  'twitter_followers 01-04-2018': 176,
  'alexa_rank 01-04-2018': 230},
 {'id': 'tether',
  'price 01-04-2018': 1.00069,
  'reddit_subscribers 01-04-2018': None,
  'twitter_followers 01-04-2018': 23302,
  'alexa_rank 01-04-2018': 34929},
 {'id': 'ripple',
  'price 01-04-2018': 0.51199,
  'reddit_subscribers 01-04-2018': 184063,
  'twitter_followers 01-04-2018': 843082,
  'alexa_rank 01-04-2018': 5224},
 {'id': 'cardano',
  'price 01-04-2018': 0.15553196120838061,
  'reddit_subscribers 01-04-2018': 60486,
  'twitter_followers 01-